# Part-of-Speech Tagging Practical Session

This part was produced using [Jupyter](http://jupyter.org).  
If you are used to it, you can [download the corresponding notebook code from here](TP3-PoSTagging.ipynb). If not, no problem at all, this is not mandatory at all: simply proceed as usual in your favorite Python environment.

## Introduction

In this practical session, our goals are to:

* better understand the role and mechanisms behind PoS tagging;
* explore applications of PoS tagging such as dealing with ambiguity or vocabulary reduction;
* get accustomed to the Viterbi algorithm through a concrete example.

This practical session is making use of the NLTK. Please refer to [this part of first practical session](TP1.php#nltk) for a setup.

## 1. Parts of Speech and Ambiguity

For this exercise, we will be using the basic functionality of the built-in PoS tagger from NLTK. All you need to know for this part can be found in [section 1  of chapter 5 of the NLTK book](http://www.nltk.org/book/ch05.html#using-a-tagger).

1. Search the web for "ambiguous headlines", to find such gems as: "_British Left Waffles on Falkland Islands_", and "_Juvenile Court to Try Shooting Defendant_". [That website (link)](http://www.ling.upenn.edu/~beatrice/humor/headlines.html) has some good ones.
1. Manually tag (by yourself) these headlines.
   Do you think the information about part-of-speech tags would help a first reading?
1. Use the built-in NLTK PoS tagger on the headlines.
   Does it find the correct tags in these cases?

**Note:** NLTK provides documentation for each tag, which can be queried using the tag, e.g.

    nltk.help.upenn_tagset('RB')

or some regular expression, e.g.

    nltk.help.upenn_tagset('NN.*')

For the rest of this practical session, you need a few nltk ressources. If not done yet (from [the first practical session](http://localhost:8888/notebooks/03-Tagging/TP1.php#nltk), please do:

In [2]:
## if not downloaded yet (see first practical session)
import os
import nltk
install_path = os.path.join(os.path.expanduser("~"), 'my_venvs', 'cs_431_env', 'lib', 'nltk_data')
print(install_path)

nltk.download('brown', download_dir = install_path)
nltk.download('universal_tagset', download_dir = install_path)
nltk.download('wordnet', download_dir = install_path)
nltk.download('tagsets', download_dir = install_path)

/home/my_venvs/cs_431_env/lib/nltk_data


[nltk_data] Downloading package brown to
[nltk_data]     /home/my_venvs/cs_431_env/lib/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/my_venvs/cs_431_env/lib/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/my_venvs/cs_431_env/lib/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /home/my_venvs/cs_431_env/lib/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [3]:
query_path = '/home/my_venvs/cs_431_env/lib/nltk_data/'
print(query_path)

nltk.data.path.append(query_path)

/home/my_venvs/cs_431_env/lib/nltk_data/


In [4]:
from nltk.corpus import brown
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

Then it might also be usefull to already import some symbols:

In [16]:
import nltk

# some headline
headline = "British Left Waffles on Falkland Islands"

[**to do:** complete the above code to tag your headline phrase]

In [ ]:
# Tokenize and then tag the headline
tagged = nltk.pos_tag( nltk.word_tokenize( headline ) );
print( tagged );
 
# some help, maybe...
for (word, tag) in tagged:
    nltk.help.upenn_tagset( tag )

In [6]:
# All possible tags in NLTK: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
print('Adverbs')
print(nltk.help.upenn_tagset('RB'))
print('Nouns')
print(nltk.help.upenn_tagset('NN.*'))
print('Verbs')
print(nltk.help.upenn_tagset('V.*'))
print('Prepositions')
print(nltk.help.upenn_tagset('RP'))

Adverbs
RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
None
Nouns
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNPS: noun, proper, plural
    Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
    Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
    Apache Apaches Apocrypha ...
NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivis

### Tagging of the headline
#### Headline: "British Left Waffles on Falkland Islands"

Manual tagging: British`Noun`, Left`Verb`, Waffles`Noun`, On`Prep`, Falkland`Noun`, Islands`Noun` <br/>
NLTK tagging: British`NNP`, Left`VBD`, Waffles`NNS`, On`RP`, Falkland`NNP`, Islands`NNS`

## 2. Vocabulary size and variability reduction

In this exercise, we will be exploring the interplay between PoS tagging, lemmatization, and vocabulary reduction.

Imagine you have a large corpus and a large number of queries of the form “_Give me all documents in which all of the words x, y and  z appear_”. One way to answer such queries efficiently is to index the documents in the corpus: for instance implement a function called `index()` for which `index(document, word)` returns non-zero if and only if word occurs in document at least once.

One way to implement this kind of index is with a matrix in which each line corresponds to a document and each column corresponds to a word. When the query above is executed, only the lines for which the entries on the x, y and z columns are simultaneously not zero are returned.

The size of this matrix is number of documents × number of distinct words that appear in the corpus. Clearly, the smaller the matrix, the faster the queries are executed. Ideally, we would like to reduce the number of words while not losing the ability to answer queries. One way to do this is to eliminate morphological variability — for instance, map both “_goes_” and “_went_” to the same column “_go_”. This is called "_lemmatization_". Part-of-speech tagging can help lemmatization so as not to confuse similar surface forms with different roles (e.g. not confuse "_can_" as in "_a can of beer_" as opposed to "_I can do it_".).

* To begin, we will need a corpus to work with. Let's make use of the [Brown Corpus](http://en.wikipedia.org/wiki/Brown_Corpus). Load the list of all word-tag tuples from the corpus:

In [7]:
brown_tagged = brown.tagged_words(tagset="universal")

and see what you got; e.g.

In [8]:
brown_tagged[0:10]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP')]

Note that we are using the “[universal](http://www.nltk.org/book/ch05.html#tab-universal-tagset)” tagset because the original Brown tagset is somewhat unwieldy.

* Next, we want to see how many distinct words appear in the corpus. Variations of the same word should be counted separately, e.g. "_go_" and "_went_" shall be counted separately here, but case should be ignored. One way to do this is to use the NLTK-supplied [`FreqDist` class](http://www.nltk.org/_modules/nltk/probability.html):

In [9]:
fdist_before = nltk.FreqDist(word.lower() for (word, tag) in brown_tagged)

This builds a frequency distribution from the words in the corpus (have a look!).
To see how many distinct words are in the distribution, simply take the length: 

In [10]:
len(fdist_before) ## should be 49815

49815

* Now run the [WordNet Lemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html) on the tagged corpus.
  Note that this lemmatizer needs information about the part of speech of a token in order to correctly lemmatize it.
  One way of lemmatizing the entire corpus is to iterate through the list of (token, tag) pairs — `brown_tagged` variable above —, lemmatize each token and add the lemma to a separate list.
  For the part-of-speech format needed for lemmatization, you might find the following function useful (do not forget to `import wordnet from nltk.corpus`):

In [11]:
# converts a pos tag from the 'universal' format to wordnet format 
def get_wordnet_pos(universal_tag):
    if universal_tag == 'VERB':
        return wordnet.VERB
    elif universal_tag == 'ADJ':
        return wordnet.ADJ
    elif universal_tag == 'ADV':
        return wordnet.ADV
    else:
        return wordnet.NOUN

* Count the distinct lemmatized words, for example using FreqDist. Do not forget to convert everything to lowercase.
  What percentage reduction did you obtain?

[**to do:** write Python code to perform lemmatization of the Brown corpus and answer the above question.]

In [12]:
# just to take a peek at the FreqDist
print(fdist_before.pprint(maxlen = 10))

# start lemmatising the corpus and add lemma to seperate list
from nltk.corpus import wordnet

lemmatiser = WordNetLemmatizer()

# Let this be a list of tuples with word, tag
lemmatised_dict = {}

for word, tag in brown_tagged:
    
    lemmatised_wrd = lemmatiser.lemmatize(word.lower(), get_wordnet_pos(tag))
    
    # Store lemmatised word in a dict for easy access later
    cur_stored_item = lemmatised_dict.get(lemmatised_wrd, {})
    
    # if lemmatised word is already in dictionary, increase count by 1
    if bool(cur_stored_item):
        cur_stored_item['cnt'] += 1
    
    # else, initialise a new dict with count of 1 and corresponding tag
    else:
        cur_stored_item['cnt'] = 1
        
    lemmatised_dict[lemmatised_wrd] = cur_stored_item

FreqDist({'the': 69971, ',': 58334, '.': 49346, 'of': 36412, 'and': 28853, 'to': 26158, 'a': 23195, 'in': 21337, 'that': 10594, 'is': 10109, ...})
None


In [13]:
print(f'No. of distinct words: {len(fdist_before)}')
print(f'No. of distinct lemmatised words: {len(lemmatised_dict)}')
print(f'Percentage reduction: {(len(fdist_before) - len(lemmatised_dict))/len(fdist_before)}')

No. of distinct words: 49815
No. of distinct lemmatised words: 39358
Percentage reduction: 0.2099166917595102


In [14]:
lemmatised_word1 = lemmatiser.lemmatize('goes', wordnet.VERB)
print(f'Lemmatised word of goes: {lemmatised_word1}')

lemmatised_word2 = lemmatiser.lemmatize('run', wordnet.VERB)
print(f'Lemmatised word of run(V): {lemmatised_word2}')

lemmatised_word3 = lemmatiser.lemmatize('run', wordnet.NOUN)
print(f'Lemmatised word of run(N): {lemmatised_word3}')

# Note: The same word with different PoS tag may be lemmatised to the same word. Not that meaningful to
# store PoS tag in dictionary

Lemmatised word of goes: go
Lemmatised word of run(V): run
Lemmatised word of run(N): run


## 3. N-gram tagging
### 3.1 Unigram tagger

In this exercise, we will see how adding context can improve the performance of automatic part-of-speech tagging. From the NLTK point of view, everything you need to know can be found in [section 5 of chapter 5 of the book](http://www.nltk.org/book/ch05.html#n-gram-tagging).

We will begin with a simple unigram tagger and build it up to a slightly more complex tagger. Unigram taggers are based on a simple statistical algorithm: for each token, assign the tag that is most likely for that particular token.

* The first thing we want to do is to consider a corpus to work with and separate it into a training set and a test set. We recommend the Brown corpus again, but this time it is a good idea to restrict it to one or two of its categories, otherwise the experiments would be too slow (we choose news and fiction categories below):


In [30]:
#import the tagged and untagged sentences
brown_tagged_sents = brown.tagged_sents(categories=['news', 'fiction'])
print("Size of corpus: {0} sentences".format(len(brown_tagged_sents)))

# split the sentences into training and test sets
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
print("Size of training set: {0} sentencces".format(size))

# Tagged test sentences (used as reference)
test_sents = brown_tagged_sents[size:]

# Untagged test sentences
raw_sents = brown.sents(categories=['news', 'fiction'])[size:]

Size of corpus: 8872 sentences
Size of training set: 7984 sentencces


* Train a unigram tagger (on the training set of sentences) and run it on some of the sentences from `raw_sents`. Observe that some words are not assigned a tag.
  Why not?
* What is the accuracy of the tagger on the test set (the [`evaluate()` function](http://www.nltk.org/api/nltk.tag.html#nltk.tag.api.TaggerI.evaluate) may come in handy here)?

[**to do:** write Python code to perform unigram tagger learning on the Brown corpus and answer the above questions.]

In [31]:
# Train a unigram tagger 
unigram_tagger = nltk.UnigramTagger(train_sents)

# Test out trained tagger on some random test sentences
print(unigram_tagger.tag(raw_sents[0]))
print(unigram_tagger.tag(raw_sents[10]))

# Reason why some words are not assigned a tag: Training data is not large enough and model has not seen some
# words before. Hence,it is not able to tag words it has not seen before

# Accuracy of tagger
print(f'Accuracy of unigram tagger: {unigram_tagger.evaluate(test_sents)}')

[('To', 'TO'), ('Mark', 'NP'), (':', ':'), ('``', '``'), ('Please', 'VB'), ('give', 'VB'), ('my', 'PP$'), ('regards', None), ('to', 'TO'), ('Myra', None), ("''", "''"), ('.', '.')]
[('``', '``'), ('Um', None), ("''", "''"), (',', ','), ('said', 'VBD'), ('the', 'AT'), ('old', 'JJ'), ('lady', 'NN'), (',', ','), ('and', 'CC'), ('brought', 'VBD'), ('her', 'PP$'), ('eyes', 'NNS'), ('down', 'RP'), ('to', 'TO'), ('the', 'AT'), ('tray', None), ('.', '.')]
Accuracy of unigram tagger: 0.8502823106037104


/tmp/ipykernel_96/3774325492.py:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of unigram tagger: {unigram_tagger.evaluate(test_sents)}')


### 3.2 Guesser

An easy way to mitigate the problem of some words not being assigned tags is to have the unigram tagger fall back on a guesser when it does not know what tag to assign to a word.

As its name suggests, a guesser is a PoS Tagger that assigns a tag to any token (be it a correct word or not). The use of a guesser as a fallback can improve the robustness of the PoS tagging system (i.e. the system will associate a PoS tag to every word, even if it was not seen beforehand, or if it is not in the lexicon).
Another scenario in which a guesser could be helpful is in the initialization phase of the Brill tagging algorithm. In addition, the guesser’s standalone performance could be used as a baseline, when evaluating more complex taggers.

For instance, perhaps the simplest guesser is the one that just assigns the same tag to every word it encounters. This is called a [“_default tagger_”](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.DefaultTagger) in NLTK.

* Instantiate a default tagger and use it to tag the test sentences (`test_sents`) defined in the previous section.
  Do not combine it yet with other taggers at this point.
  Report the accuracy. (Feel free to look at the examples in [sections 4](http://www.nltk.org/book/ch05.html#automatic-tagging)  [and 5](http://www.nltk.org/book/ch05.html#n-gram-tagging)  from chapter 5 of the NLTK book.)

In [32]:
# Get the most frequent tag in our train dataset
from itertools import chain

words_and_tags = list(chain.from_iterable(train_sents))
tags = [tag for (word,tag) in words_and_tags]
most_freq_tag = nltk.FreqDist(tags).max()

default_tagger = nltk.DefaultTagger(most_freq_tag)
print(f'Accuracy of default tagger: {default_tagger.evaluate(test_sents)}')

Accuracy of default tagger: 0.11230377861884966


/tmp/ipykernel_96/2981797804.py:9: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of default tagger: {default_tagger.evaluate(test_sents)}')


* Then use the default tagger as a fallback for a Unigram tagger. Report the change in accuracy.

In [34]:
# Get most frequent tag in corpus
from itertools import chain

words_and_tags = list(chain.from_iterable(train_sents))
tags = [tag for (word,tag) in words_and_tags]
most_freq_tag = nltk.FreqDist(tags).max()

# Instantiate the default tagger
default_tagger = nltk.DefaultTagger(most_freq_tag)

# Instantiate the unigram tagger which uses the default tagger as a fallback
unigram_with_default_tagger = nltk.UnigramTagger(train_sents, backoff = default_tagger)

# Report the accuracy

# Accuracy of tagger
print(f'Accuracy of unigram tagger: {unigram_with_default_tagger.evaluate(test_sents)}')

# ----------------------------------------------------------
# Extra: Instantiate a bigram which uses unigram as a backoff
bigram_tagger = nltk.BigramTagger(train_sents, backoff = unigram_tagger)
# Accuracy of tagger
print(f'Accuracy of bigram tagger: {bigram_tagger.evaluate(test_sents)}')

/tmp/ipykernel_96/3834158350.py:17: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of unigram tagger: {unigram_with_default_tagger.evaluate(test_sents)}')


Accuracy of unigram tagger: 0.868834150276106
Accuracy of bigram tagger: 0.866848669107154


/tmp/ipykernel_96/3834158350.py:23: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of bigram tagger: {bigram_tagger.evaluate(test_sents)}')


* How would you build a better guesser, while still not relying on any further training data?

A: Make use of longer n-grams where n > 1. This allows for more dependencies between the words to be captured and lead to better accuracy while not requiring more training data. Also, default taggers/ simpler n-gram models can be used as the fall back when the lead model is not able to assign a tag.

One step further than the default tagger could be to assign tags to tokens on the basis of matching patterns. In other words, the form of the token is considered when guessing its part-of-speech. For example, it might guess that any word ending in “ed” is the past participle of a verb, and any word ending with “'s” is a possessive noun. This is called a [“_regular expression tagger_”](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.RegexpTagger) and is already available in NLTK.

* Instantiate a regular expression tagger and use it to tag the test sentences. You could use the same rules provided in the link above, or define some other rules of your own.
  Do not combine it yet with other taggers at this point.
  Report the accuracy.
* Now use the regular expression tagger as a fallback for a Unigram tagger.
  Report the change in accuracy.

In [23]:
# Instantiate a regular expression tagger
patterns = [
        (r'.*ing$', 'VBG'),                # gerunds
        (r'.*ed$', 'VBD'),                 # simple past
        (r'.*es$', 'VBZ'),                 # 3rd singular present
        (r'.*ould$', 'MD'),                # modals
        (r'.*\'s$', 'NN$'),                # possessive nouns
        (r'.*s$', 'NNS'),                  # plural nouns
        (r'^-?[0-9]+(\.[0-9]+)?$', 'CD'),  # cardinal numbers
        (r'.*', 'NN')                      # nouns (default)
    ]

regexp_tagger = nltk.RegexpTagger(patterns)

print(f'Accuracy of Reg Expression tagger: {regexp_tagger.evaluate(test_sents)}')

/tmp/ipykernel_96/1215325866.py:15: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of Reg Expression tagger: {regexp_tagger.evaluate(test_sents)}')


Accuracy of Reg Expression tagger: 0.18303654526276603


In [35]:
# Use egexp_tagger as a fallback for unigram tagger

unigram_with_regexp_tagger = nltk.UnigramTagger(train_sents, backoff = regexp_tagger)
print(f'Accuracy of Unigram tagger: {unigram_with_regexp_tagger.evaluate(test_sents)}')

Accuracy of Unigram tagger: 0.8824222870261215


/tmp/ipykernel_96/1913435978.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of Unigram tagger: {unigram_with_regexp_tagger.evaluate(test_sents)}')


### 3.3 Bigrams and More

When performing PoS tagging using a unigram-based tagger, the larger context in which the words appear is ignored. Unigram taggers consider only one item of context, in isolation. This poses a significant limitation on any tagger in this class: the most that it can do is to assign the a priori most likely tag to a given token. In a sense, the tag of each word is decided before it is placed in a sentence.  Therefore, the word “_object_” would be assigned the same part of speech every time, regardless if it appears as “_the object_” (should be a noun) or as “_to object_” (should be a verb).  

An n-gram tagger picks the tag that is most likely in the given context of size n. An instance of an n-gram tagger is the bigram tagger, which considers groups of two tokens when deciding on the parts-of-speech. For example, we would expect a bigram tagger to find the correct tags for the two occurrences of the word “_objec_t” in the following sentence (as opposed to a unigram tagger):

“_I did not object to the object_”

Indeed, a NLTK unigram tagger trained on the Brown corpus yields: 

In [27]:
install_path = os.path.join(os.path.expanduser("~"), 'my_venvs', 'cs_431_env', 'lib', 'nltk_data')
print(install_path)

nltk.download('punkt', download_dir = install_path)

/home/my_venvs/cs_431_env/lib/nltk_data


[nltk_data] Downloading package punkt to
[nltk_data]     /home/my_venvs/cs_431_env/lib/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
sentence = "I did not object to the object"
tokens =  nltk.word_tokenize(sentence)

print(unigram_tagger.tag(tokens))
print(unigram_with_default_tagger.tag(tokens))
print(unigram_with_regexp_tagger.tag(tokens))

[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]
[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]
[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]


No way to have a different tag for the two "_object_" words.

* Train a bigram tagger alone on the training set (`train_sents`) and check it in the above sentence (weird result expected, see below).
* Then, find its accuracy on the test set (`test_sents`).

In [37]:
# Tag the sentence using bigram tagger and see the output
bigram_tagger = nltk.BigramTagger(train_sents)

print(bigram_tagger.tag(tokens))

# Find out the accuracy on the test set
print(f'Accuracy of bigram tagger: {bigram_tagger.evaluate(test_sents)}')

[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', None), ('object', None)]
Accuracy of bigram tagger: 0.19631445058013278


/tmp/ipykernel_96/68309471.py:7: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of bigram tagger: {bigram_tagger.evaluate(test_sents)}')


You have probably found a surprising result on our sample sentence and very low accuracy for the bigram tagger, when run alone. If you run the trained bigram tagger on a sentence it has not seen during training (e.g. bigram.tag(raw_sents[1]), you will notice that a majority of words will be tagged as “None”. When the bigram tagger encounters a new word, it is unable to assign a tag. It will also be unable to tag the following word — even if it was seen during training — because it was never seen preceded by a “None” tag. From this point on, all the words up to the end of the sentence will be tagged as “None” for the same reason. This explains the low accuracy of the bigram tagger when run alone.

The problem that arises when using n-grams is that once we pass to larger values of n, more data is needed for training, as the specificity of the context increases. In other words, there are many more possible bigrams than unigrams and thus the word/tag combinations that we need to consider when tagging are less likely to have appeared in the training set, as n increases. This is known as the sparse data problem, and is quite pervasive in NLP.

This means that, when using n-gram taggers, we encounter the following tradeoff: having n large means more accurate predictions, at the cost of having to provide more training data; having a smaller n, leads to not so accurate predictions, but less necessary training data.

In order to overcome this limitation, we can train a bigram tagger that falls back on our “unigram+default” or “unigram+regexp” taggers defined in the previous section.

* Try this out now (an example for a bigram tagger with fallback on “unigram+default” can be found in the book, [section 5.5.4 “_Combining Taggers_”](http://www.nltk.org/book/ch05.html#combining-taggers)). Does this improve performance?
* What about adding a trigram tagger that falls back on the “bigram+unigram+default” or on “bigram+unigram+regexp”?  

In [38]:
# Bigram tagger with unigram + default
bigram_tagger = nltk.BigramTagger(train_sents, backoff = unigram_with_default_tagger)
print(f'Accuracy of bigram tagger with unigram+default fallback: {bigram_tagger.evaluate(test_sents)}')

# Trigram tagger with bigram + unigram + default
trigram_tagger = nltk.TrigramTagger(train_sents, backoff = bigram_tagger)
print(f'Accuracy of trigram tagger with bigram+unigram+default fallback: {trigram_tagger.evaluate(test_sents)}')

# Trigram tagger with trigram + bigram + unigram + regexp
bigram_tagger = nltk.BigramTagger(train_sents, backoff = unigram_with_regexp_tagger)
trigram_tagger = nltk.TrigramTagger(train_sents, backoff = bigram_tagger)
print(f'Accuracy of trigram tagger with bigram+unigram+default fallback: {trigram_tagger.evaluate(test_sents)}')

/tmp/ipykernel_96/1075774205.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of bigram tagger with unigram+default fallback: {bigram_tagger.evaluate(test_sents)}')


Accuracy of bigram tagger with unigram+default fallback: 0.8848420922007818


/tmp/ipykernel_96/1075774205.py:7: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of trigram tagger with bigram+unigram+default fallback: {trigram_tagger.evaluate(test_sents)}')


Accuracy of trigram tagger with bigram+unigram+default fallback: 0.8845939070546628
Accuracy of trigram tagger with bigram+unigram+default fallback: 0.898492275237327


/tmp/ipykernel_96/1075774205.py:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(f'Accuracy of trigram tagger with bigram+unigram+default fallback: {trigram_tagger.evaluate(test_sents)}')


## 4. Viterbi Algorithm

Code the Viterbi algorithm (in your most favorite programming language) and apply it to the toy example provided on slide 8 of the lecture on HMM: two symbols (H,T), two states (the two coins),  
A = [[ 0.4 0.6 ]; [0.9 0.1 ]],  
B = [[ 0.49 0.51 ]; [ 0.85 0.15 ]],  
I = [ 0.5 0.5 ].

What is (one of) the most probable sequence of states corresponding to the observation `HTTHTTHHTTHTTTHHTHHTTHTTTTHTHHTHTHHTTTH`?

In [6]:
solns = ['211211121121111211211211112112121121112',
'211211121121111211211211112112121211112',
'211211121121111211211211112121121121112',
'211211121121111211211211112121121211112',
'211211121121111212111211112112121121112',
'211211121121111212111211112112121211112',
'211211121121111212111211112121121121112',
'211211121121111212111211112121121211112',
'211211121121112111211211112112121121112',
'211211121121112111211211112112121211112',
'211211121121112111211211112121121121112',
'211211121121112111211211112121121211112',
'211211121121112112111211112112121121112',
'211211121121112112111211112112121211112',
'211211121121112112111211112121121121112',
'211211121121112112111211112121121211112',
'211211211121111211211211112112121121112',
'211211211121111211211211112112121211112',
'211211211121111211211211112121121121112',
'211211211121111211211211112121121211112',
'211211211121111212111211112112121121112',
'211211211121111212111211112112121211112',
'211211211121111212111211112121121121112',
'211211211121111212111211112121121211112',
'211211211121112111211211112112121121112',
'211211211121112111211211112112121211112',
'211211211121112111211211112121121121112',
'211211211121112111211211112121121211112',
'211211211121112112111211112112121121112',
'211211211121112112111211112112121211112',
'211211211121112112111211112121121121112',
'211211211121112112111211112121121211112']

In [13]:
def test_soln(soln):
    
    soln = [str(sol) for sol in soln]
    soln = ''.join(soln)
    
    if soln in solns:
        print('Solution is correct')
    else:
        print('Solution is wrong')

In [5]:
import numpy as np
observations = 'HTTHTTHHTTHTTTHHTHHTTHTTTTHTHHTHTHHTTTH'
observations_idxed = [0 if obsv == 'H' else 1 for obsv in observations]

# Initial probabilities
# [p_1 = 0.5, p_2 = 0.5]
I = np.array([0.5, 0.5])

# Transition Probabilities
# [[state1->1, state1->2], [state2->1, state2->2]]
A = np.array([[0.4, 0.6], [0.9, 0.1]])

# Emission Probabilites
# [[state1_head, state1_tail], [state2_head, state2_tail]]
B = np.array([[0.49, 0.51], [0.85, 0.15]])

# Array to store results
res = np.zeros((len(observations,)), dtype = np.int32)

# Number of possible states per observation
N_states = I.shape[0]
N_observations = len(observations)

# numpy array to store probabilities
probs = np.zeros((N_observations, N_states), dtype = np.float64)

# array to store sequence of most probable states
state_arr = []

# Initialise initial probabilities
first_obsv = observations_idxed[0]
for j in range(N_states):
    probs[0,j] = I[j]*B[j][first_obsv]

# Mark the most probable path
# +1 since probs is 0 indexed while the states are 1 indexed
state_arr.append(np.argmax(probs[0,:]) + 1)

# for 2nd observation to nth observation,
for i in range(1, N_observations):
        
        # Get previous state
        prev_state = state_arr[i-1] - 1
        prev_prob = np.amax(probs[(i-1),:])
        
        # for each state, find out the probability
        for j in range(N_states):
            
            probs[i,j] = B[j][observations_idxed[i]]*A[prev_state][j]*np.amax(prev_prob)
            
        # Find out most probable state given observation
        current_state = np.argmax(probs[i,:])
        
        # Mark the most probable path
        state_arr.append(current_state + 1)
        
# Return the most probable sequence of states
print(state_arr)

[2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2]


In [14]:
test_soln(state_arr)

Solution is correct
